In [1]:
import numpy as np
import igl
import meshplot as mp

In [2]:
#global configuration
n = 32 #resolution
normalizationmethod=1 # 0 for "obj fit the (-1,1) box" by normalizing obj. 1 for "box fit the obj closely by constructing a streched grid.
if normalizationmethod==0:
    wendlandRadius=0.2
    point_size=0.1 #in plot
    gridlength=0.4
elif normalizationmethod==1:
    wendlandRadius=13
    point_size=6
    gridlength=20
k=0
coefficientnumber=int(np.array([1,4,10])[k])
debug=True
spatial=True
if debug==True:
    import time

In [3]:
# Utility function to generate a tet grid
# n is a 3-tuple with the number of cell in every direction
# mmin/mmax are the grid bounding box corners

def tet_grid(n, mmin, mmax):
    nx = n[0]
    ny = n[1]
    nz = n[2]
    
    delta = mmax-mmin
    
    deltax = delta[0]/(nx-1)
    deltay = delta[1]/(ny-1)
    deltaz = delta[2]/(nz-1)
    
    T = np.zeros(((nx-1)*(ny-1)*(nz-1)*6, 4), dtype=np.int64)
    V = np.zeros((nx*ny*nz, 3))

    mapping = -np.ones((nx, ny, nz), dtype=np.int64)


    index = 0
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                mapping[i, j, k] = index
                V[index, :] = [i*deltax, j*deltay, k*deltaz]
                index += 1
    assert(index == V.shape[0])
    
    tets = np.array([
        [0,1,3,4],
        [5,2,6,7],
        [4,1,5,3],
        [4,3,7,5],
        [3,1,5,2],
        [2,3,7,5]
    ])
    
    index = 0
    for i in range(nx-1):
        for j in range(ny-1):
            for k in range(nz-1):
                indices = [
                    (i,   j,   k),
                    (i+1, j,   k),
                    (i+1, j+1, k),
                    (i,   j+1, k),

                    (i,   j,   k+1),
                    (i+1, j,   k+1),
                    (i+1, j+1, k+1),
                    (i,   j+1, k+1),
                ]
                
                for t in range(tets.shape[0]):
                    tmp = [mapping[indices[ii]] for ii in tets[t, :]]
                    T[index, :]=tmp
                    index += 1
                    
    assert(index == T.shape[0])
    
    V += mmin
    return V, T

# Implementing a spatial index to accelerate neighbor calculations

In [4]:
# Implementing a spatial index to accelerate neighbor calculations        
def findgridcell(point,bbox_min,gridlength): #bbox_min (1,3) ndarray. point is position
    return ((point-bbox_min)/gridlength).astype(int)  # (,3) ndarray
    
def constructspatialgrid(points,bbox_min,bbox_max,gridlength):  #points is the obj. Pure cube grid. should contain all obj&tet_grid points. length is determined by wendlandradius
    gridcellnumbers=findgridcell(bbox_max,bbox_min,gridlength)+1
    if debug==True:
        print("gridcellnumbers"+str(gridcellnumbers))
    gridcells=[]        #shape=(gridcellsnumbers[0],gridcellsnumbers[1],gridcellsnumbers[2],*)
    for i in range(gridcellnumbers[0]):
        gridcells.append([])
        for j in range(gridcellnumbers[1]):
            gridcells[i].append([])
            for k in range(gridcellnumbers[2]):
                gridcells[i][j].append([])
    for (i,point) in enumerate(points):
        index=findgridcell(point,bbox_min,gridlength)
        gridcells[index[0]][index[1]][index[2]].append(i)
    return gridcells,gridcellnumbers

#def spatialgrid_add(pointindex,points,bbox_min,gridcells,gridcellnumbers,gridlength):
    #index=findgridcell(points[pointindex],bbox_min,gridlength)
    #gridcells[index[0]][index[1]][index[2]].append(pointindex)
    
def find_closed_point_spatial(point,points,bbox_min,gridcells,gridcellnumbers,gridlength): # point is position. points is positions of obj constraints
    cell=findgridcell(point,bbox_min,gridlength)
    adjcelllist=[]
    for i in range(-1,2):
        for j in range(-1,2):
            for k in range(-1,2):
                i1,i2,i3=cell[0]+i,cell[1]+j,cell[2]+k
                if i1>=0 and i1<gridcellnumbers[0] and i2>=0 and i2<gridcellnumbers[1] and i3>=0 and i3<gridcellnumbers[2]:
                    adjcelllist.extend(gridcells[i1][i2][i3])
    adjcelllist=np.array(adjcelllist) #可能是空列表
    return adjcelllist[find_closed_point(point, points[adjcelllist])]

def closest_points_spatial(point,points,bbox_min,gridcells,gridcellnumbers,gridlength,wendlandRadius): # point is position. Return like closest_points
    cell=findgridcell(point,bbox_min,gridlength)
    adjcelllist=[]
    for i in range(-1,2):
        for j in range(-1,2):
            for k in range(-1,2):
                i1,i2,i3=cell[0]+i,cell[1]+j,cell[2]+k
                if i1>=0 and i1<gridcellnumbers[0] and i2>=0 and i2<gridcellnumbers[1] and i3>=0 and i3<gridcellnumbers[2]:
                    adjcelllist.extend(gridcells[i1][i2][i3])
    #print("adjcelllist"+str(adjcelllist))
    adjcelllist=np.array(adjcelllist,dtype=int)
    indexinadjlist = closest_points(point,points[adjcelllist],wendlandRadius).squeeze(axis=1)
    #print("indexinadjlist"+str(indexinadjlist))
    return np.expand_dims(adjcelllist[indexinadjlist],axis=1)
    
def normalize(points,min,max):
    return ((points-np.min(points,axis=0))/(np.max(np.max(points,axis=0)-np.min(points,axis=0))))*(max-min)-(max-min)/2

def find_closed_point(point, points): #assume the return is always a int
    distance=np.linalg.norm(points-point,axis=1)
    return np.argmin(distance)

def closest_points(point, points, h):
    distance=np.linalg.norm(points-point,axis=1)
    return np.argwhere(distance < h).astype(int)


# Reading point cloud

In [5]:
pi, v = igl.read_triangle_mesh("data/cat.off")
pi /= 10
ni = igl.per_vertex_normals(pi, v)
if debug==True: 
    mp.plot(pi, shading={"point_size": point_size})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

In [6]:
if normalizationmethod==0:
    bbox_min = np.array([-1., -1., -1.])
    bbox_max = np.array([1., 1., 1.])
    bbox_diag = np.linalg.norm(bbox_max - bbox_min)
    
    npi=normalize(pi,np.min(bbox_min),np.max(bbox_max))
    #npiplus=normalize(piplus,np.min(bbox_min),np.max(bbox_max))
    #npiminus=normalize(piminus,np.min(bbox_min),np.max(bbox_max))

    bbox_min=bbox_min - 0.05 * bbox_diag
    bbox_max=bbox_max + 0.05 * bbox_diag


if normalizationmethod==1:
    bv,bf=igl.bounding_box(pi)
    bbox_min=np.min(np.array(bv),axis=0)
    bbox_max=np.max(np.array(bv),axis=0)
    bbox_diag = np.linalg.norm(bbox_max - bbox_min)
    bbox_min=bbox_min - 0.05 * bbox_diag
    bbox_max=bbox_max + 0.05 * bbox_diag
    if debug==True:
        print(bbox_min)
        print(bbox_max)
    #p.add_points(x,c=np.ones_like(x)*np.array([0.5, 0.5, 0.5]), shading={"point_size": 2})
    npi=pi
    #npiplus=piplus
    #npiminus=piminus
if debug==True:
    print(np.min(npi))
    print(np.max(npi))

if spatial==True:       
    gridcells,gridcellnumbers= constructspatialgrid(npi,bbox_min,bbox_max,gridlength)
    if debug==True:
        print(gridcells)

[ -29.86176654  -70.86176654 -106.16176654]
[39.86176654 23.86176654 12.96176654]
-99.0
32.7
gridcellnumbers[4 5 6]
[[[[224, 248, 249, 250], [], [], [], [], []], [[216, 219, 220, 221, 222, 223, 242, 243, 244, 245, 246, 247], [213, 214, 215, 217, 218], [225, 226], [], [], []], [[193, 199, 200, 202, 203, 204, 205, 206, 207, 208, 209, 240, 241], [195, 196, 210, 211, 212], [194], [], [29, 30], []], [[], [], [], [82, 83, 94, 95, 96, 105], [31, 32, 33, 34, 35, 36, 54, 55, 56, 57, 58, 59, 60, 61, 64, 65, 66, 67, 68, 69, 70, 71, 73, 74, 75, 76, 77, 78, 79, 80, 81, 92, 93, 300], []], [[], [], [], [], [62], []]], [[[251, 252, 253, 254, 256, 257, 260, 261, 265, 348], [232, 234, 235, 255, 258, 263], [229, 233, 236], [], [], []], [[], [], [181, 227, 228, 230, 237], [179, 231], [], []], [[192, 198, 201], [186, 197], [121, 124, 125, 126, 127, 180, 182, 183, 184, 185], [89, 117, 118, 119, 120, 177], [22, 24, 28, 72, 84, 86, 112, 113, 115, 175], [10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 23, 25]], [[

In [7]:
eps=igl.bounding_box_diagonal(npi)*0.01
piplus=np.zeros_like(npi)
piminus=np.zeros_like(npi)
for (i,point) in enumerate(npi):
    temp=point+eps*ni[i]
    neweps=eps
    while True:
        if spatial==False:
            closest=find_closed_point(temp,npi)
        else:
            closest=find_closed_point_spatial(temp,npi,bbox_min,gridcells,gridcellnumbers,gridlength)
        if closest==i:
            break
        if debug==True:
            test1=find_closed_point(temp,npi)
            test2=find_closed_point_spatial(temp,npi,bbox_min,gridcells,gridcellnumbers,gridlength)
            if test1!=test2:
                print(findgridcell(temp,bbox_min,gridlength))
                print(test1)
                print(test2)
                raise Exception("Design wrong!!!")
        neweps=neweps/2
        temp=point+neweps*ni[i]
    piplus[i]=temp
for (i,point) in enumerate(npi):
    temp=point-eps*ni[i]
    neweps=eps
    while True:
        if spatial==False:
            closest=find_closed_point(temp,npi)
        else:
            closest=find_closed_point_spatial(temp,npi,bbox_min,gridcells,gridcellnumbers,gridlength)
        if closest==i:
            break
        if debug==True:
            test1=find_closed_point(temp,npi)
            test2=find_closed_point_spatial(temp,npi,bbox_min,gridcells,gridcellnumbers,gridlength)
            if test1!=test2:
                print(findgridcell(temp,bbox_min,gridlength))
                print(test1)
                print(test2)
                raise Exception("Design wrong!!!")
        neweps=neweps/2
        temp=point-neweps*ni[i]
    piminus[i]=temp
#程序改动：原来是pi和piplus一起算出来后normal。现在是pi normal后算的piplus，所以piplus本身是normal的
npiplus=piplus
npiminus=piminus
print("The following plot is the required output for Setting up the Constraints section")
p=mp.plot(npi,c=np.ones_like(npi)*np.array([0, 0, 1]), shading={"point_size": point_size})
p.add_points(piplus,c=np.ones_like(npi)*np.array([1, 0, 0]), shading={"point_size": point_size})
p.add_points(piminus,c=np.ones_like(npi)*np.array([0, 1, 0]), shading={"point_size": point_size})



#######临时测试
#list=[590,614,616]
#list=[246, 247, 248, 249, 250, 218, 219, 223, 224, 251, 253, 256, 257]
#p.add_points(np.concatenate((npi,npiplus,npiminus))[list],shading={"point_size": point_size*2})

The following plot is the required output for Setting up the Constraints section


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

2

# MLS function

In [8]:
x,T=tet_grid([n,n,n],bbox_min,bbox_max)

In [9]:
if debug==True:
    p=mp.plot(npi,v)
    p.add_points(x,shading={"point_size": point_size})


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

In [10]:
def wendlandweight(p1,p2,h):
    distance=np.linalg.norm(p1-p2)
    return (1-distance/h)**4 * (4*distance/h+1)

def functioninvector_k2(x,y,z):
    return np.array([1,x,y,z,x**2,x*y,x*z,y**2,y*z,z**2])

def functioninvector_k1(x,y,z):
    return np.array([1,x,y,z])

def functioninvector_k0(x,y,z):
    return np.array([1])
    
def functioninvector(x,y,z,k):
    if k==2:
        return functioninvector_k2(x,y,z)
    elif k==1:
        return functioninvector_k1(x,y,z)
    elif k==0:
        return functioninvector_k0(x,y,z)

In [15]:
funcargs=np.zeros((x.shape[0],coefficientnumber)) #1,x,y,z,x^2,xy,xz,y^2,yz,z^2
points=np.concatenate((npi,npiplus,npiminus))
if spatial==True:       
    gridcells,gridcellnumbers= constructspatialgrid(points,bbox_min,bbox_max,gridlength)
fx=np.zeros((x.shape[0],1))
if debug==True:
    test1sum=0
    test2sum=0
for (i,xi) in enumerate(x): 
    if spatial==False:
        adjlist=np.array(closest_points(xi,points,wendlandRadius)).squeeze(axis=1)
    else:
        adjlist=np.array(closest_points_spatial(xi,points,bbox_min,gridcells,gridcellnumbers,gridlength,wendlandRadius)).squeeze(axis=1)  
    if debug==True:
        timetest1start=time.time()
        test1=np.array(closest_points(xi,points,wendlandRadius)).squeeze(axis=1)
        timetest1end=time.time()
        timetest2start=time.time()
        test2=np.array(closest_points_spatial(xi,points,bbox_min,gridcells,gridcellnumbers,gridlength,wendlandRadius)).squeeze(axis=1) 
        timetest2end=time.time()
        test1sum+=timetest1end-timetest1start
        test2sum+=timetest2end-timetest2start
        if i==100:
            print("non-spatial time 2th:"+str(timetest1end-timetest1start))
            print("spatialtime 2th:"+str(timetest2end-timetest2start))
        if np.array_equal(test1,np.sort(test2))!=True:
            print(test1)
            print(test2)
            print("Design wrong!!!")
    a=np.zeros((adjlist.size,coefficientnumber))
    b=np.zeros((adjlist.size,1))
    w=np.zeros((adjlist.size,adjlist.size)) #diagonal matrix
    #print(adjlist.shape) if adjlist.shape[0]!=0 else None
    if(len(adjlist)<coefficientnumber):
        fx[i]=10000
        continue
    #print(adjlist)

    for (j,adj) in enumerate(adjlist):
        adj_position=points[adj]
        originalpiindex=adj % (npi.shape[0])
        originalpi_position=points[originalpiindex]
        eps=(adj_position-originalpi_position)[0]/ni[originalpiindex,0]
        a[j,:]=functioninvector(adj_position[0],adj_position[1],adj_position[2],k)
        b[j]=eps  #what would be range?
        w[j,j]=wendlandweight(adj_position,xi,wendlandRadius)
        #print(a[j,:])
        #print(b[j])
        #print(w[j,j])
    funcargs[i]=np.linalg.solve(a.T@w@a,a.T@w@b).T
    fx[i]=(functioninvector(xi[0],xi[1],xi[2],k)@funcargs[i]).T

if debug==True:
    print("total time on test1:"+str(test1sum))
    print("total time on test2:"+str(test2sum))

gridcellnumbers[4 5 6]
non-spatial time 2th:0.0
spatialtime 2th:0.0
total time on test1:1.6663627624511719
total time on test2:2.8917653560638428


In [12]:
# Treshold fx to visualize inside outside
ind = np.zeros_like(fx)
ind[fx >= 0] = 1 #yellow
ind[fx < 0] = -1 # black
print("The following plot is the required output for \"Use MLS interpolation to extend to function f\" section")
p=mp.plot(x, c=ind, shading={"point_size": point_size,"width": 800, "height": 800})
if debug==True and False:
    p.add_points(np.expand_dims(x[1],0),shading={"point_size":30})
    p.add_points(x,shading={"point_size":10})

The following plot is the required output for "Use MLS interpolation to extend to function f" section


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

# Marching to extract surface

In [13]:
# Marcing tet to extract surface
sv, sf, _, _ = igl.marching_tets(x, T, fx, 0)
components=igl.facet_components(sf)
index,count=np.unique(components,return_counts=True)
print(index)
print(count)
index=np.where(count==np.max(count))[0][0]  #argwhere?
print(index)
filteredf=sf[components==index]
mp.plot(sv, filteredf, shading={"wireframe": True})

[0]
[24234]
0


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(4.8865900…

# Experienment with parameters
All the blocks above shows a reconstruction with fixed parameters.
Below we experientment with different configuration. 

In [14]:
def myimplicitandreconstruct(x,T,npi,npiplus,npiminus,k,coefficientnumber,n,wendlandRadius): #functioninvector
    funcargs=np.zeros((x.shape[0],coefficientnumber)) #1,x,y,z,x^2,xy,xz,y^2,yz,z^2
    points=np.concatenate((npi,npiplus,npiminus))
    fx=np.zeros((x.shape[0],1))
    for (i,xi) in enumerate(x): 
        adjlist=np.array(closest_points(xi,points,wendlandRadius)).squeeze(axis=1)
        a=np.zeros((adjlist.size,coefficientnumber))
        b=np.zeros((adjlist.size,1))
        w=np.zeros((adjlist.size,adjlist.size)) #diagonal matrix
        #print(adjlist.shape) if adjlist.shape[0]!=0 else None
        if(len(adjlist)<coefficientnumber):
            fx[i]=10000
            continue
        #print(adjlist)

        for (j,adj) in enumerate(adjlist):
            adj_position=points[adj]
            originalpiindex=adj % (npi.shape[0])
            originalpi_position=points[originalpiindex]
            eps=(adj_position-originalpi_position)[0]/ni[originalpiindex,0]
            a[j,:]=functioninvector(adj_position[0],adj_position[1],adj_position[2],k)
            b[j]=eps  #what would be range?
            w[j,j]=wendlandweight(adj_position,xi,wendlandRadius)
            #print(a[j,:])
            #print(b[j])
            #print(w[j,j])
        funcargs[i]=np.linalg.solve(a.T@w@a,a.T@w@b).T
        fx[i]=(functioninvector(xi[0],xi[1],xi[2],k)@funcargs[i]).T
    sv, sf, _, _ = igl.marching_tets(x, T, fx, 0)
    components=igl.facet_components(sf)
    index,count=np.unique(components,return_counts=True)
    index=np.where(count==np.max(count))[0][0]
    filteredf=sf[components==index]
    return sv,filteredf